In [1]:
import plcoding
from plcoding.cpp_core.monotone import MCListDecoder
import numpy as np
from tqdm import trange

In [2]:
# 先验概率分布
src_bases = np.array([2, 2], dtype=int); src_probs = np.array([0.01, 0.10, 0.09, 0.80])
#src_bases = np.array([2, 2], dtype=int); src_probs = np.array([0.11, 0.00, 0.00, 0.89])
#src_bases = np.array([2], dtype=int); src_probs = np.array([0.11, 0.89])
n_var = src_bases.size
code_lvl = 3
code_len = (1 << code_lvl)
# 生成一条简单的单调链
monotone_chain = []
for i in range(n_var):
    monotone_chain.append(np.zeros((code_len,), dtype=int) + i)
monotone_chain = np.concatenate(monotone_chain)
# 初始化译码器
list_size = 4
decoder = MCListDecoder(code_len, src_bases, list_size)
# 生成一批测试数据
decoder.set_priors(np.tile(src_probs, (code_len, 1)))

In [3]:
decoder.reset()
headers = np.zeros(n_var, dtype=int)
for sup in trange(len(monotone_chain), ncols=80):
    var = monotone_chain[sup]
    index = headers[var]
    headers[var] += 1
    decoder.explore_at(var, index)

100%|███████████████████████████████████████| 16/16 [00:00<00:00, 122461.43it/s]


In [4]:
decoder.get_results().reshape([-1, code_len, n_var])

array([[[1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1]],

       [[1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 0],
        [1, 1],
        [1, 1],
        [1, 1]],

       [[1, 1],
        [1, 1],
        [1, 0],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1]],

       [[1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 0],
        [1, 1]]], dtype=int32)

In [5]:
decoder.get_likelihoods()

array([-1.78514841, -3.96995047, -3.96995047, -3.96995047])